In [1]:
# imports
import sys, os
from pathlib import Path
from tqdm import tqdm
import pickle

import numpy as np
import pandas as pd

from brainbox.metrics import single_units as qm

In [85]:
# path definitions
# maybe it's possible to abbreviate this
exp_folder = Path('/media/georg/htcondor/shared-paton/georg/DAtime/data/batch_24a/2024-06-06_JJP-08672_dh_1-6-1')
run_folder = exp_folder / 'stim_run_1_g0'
imec_bin_path = run_folder / 'stim_run_1_g0_t0.imec0.ap.bin'
ni_bin_path = run_folder / 'stim_run_1_g0_t0.nidq.bin'
ks_folder = run_folder / 'ibl_sorter_results'
results_folder = ks_folder / 'results'

In [2]:
exp_folder = Path('/media/georg/htcondor/shared-paton/georg/DAtime/data/batch_24a_sorted/2024-06-08_JJP-08628_dh_5-6-1/')
run_folder = exp_folder / 'stim_run_3_g0'
imec_bin_path = run_folder / 'stim_run_3_g0_t0.imec0.ap.bin'
ni_bin_path = run_folder / 'stim_run_3_g0_t0.nidq.bin'
ks_folder = run_folder / 'ibl_sorter_results'
results_folder = ks_folder / 'results'

In [3]:
spike_times = np.load(ks_folder / 'spike_times.npy')
spike_clusters = np.load(ks_folder / 'spike_clusters.npy')
spike_amps = np.load(ks_folder / 'amplitudes.npy')

In [88]:
# # calculate spike depth
# # using the absolute amplitude a templates on each channel for a weighted sum

# # the xy coordinates of the channels
# channel_positions = np.load(ks_folder / 'channel_positions.npy')

# # templates = n_templates x n_timesample x n_channels
# templates = np.load(ks_folder / 'templates.npy')

# templates_depths = np.empty(templates.shape[0])
# for i in range(templates.shape[0]):
#     T = templates[i,:,:]
#     W = np.max(np.absolute(T), axis=0) # the weights
#     W = W / np.sum(W)
#     channel_depths = channel_positions[:,1]
#     templates_depths[i] = np.sum(channel_depths * W)

# spike_depths = templates_depths[spike_clusters]

In [4]:
from ibllib.ephys.ephysqc import phy_model_from_ks2_path
m = phy_model_from_ks2_path(ks_folder, bin_path=imec_bin_path)
cluster_ids = m.spike_clusters
ts = m.spike_times
amps = m.amplitudes
depths = m.depths
r = qm.quick_unit_metrics(cluster_ids, ts, amps, depths)
MetricsDf = pd.DataFrame(r)

/home/georg/anaconda3/envs/ephys/lib/python3.10/site-packages/phylib/io/model.py:1083: RuntimeWarning: invalid value encountered in divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /


In [5]:
# get UnitsDf and add the metrics
UnitsDf = pd.read_csv(results_folder / 'UnitsDf.csv')
UnitsDf = pd.merge(MetricsDf, UnitsDf, on='cluster_id')

# and store
UnitsDf.to_csv(ks_folder / 'results' / 'UnitsDf.csv', index=None)

In [123]:
# do something with the metrics
import seaborn as sns
import matplotlib.pyplot as plt
for col in UnitsDf.columns:
    fig, axes = plt.subplots()
    sns.histplot(UnitsDf, x=col)

/tmp/ipykernel_3911944/1791082637.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots()
